In [10]:
import os
import glob
import pyart
import numpy as np
import dp_libs
import hsda
import hdr
import mesh
from datetime import datetime

In [13]:
#paths
vol_path         = '/g/data1a/rq0/odim_archive/hsda-testing/20141127_mdv'
out_path         = '/g/data1a/rq0/odim_archive/hsda-testing/20141127_processed'
sonde_ffn        = '/g/data1a/rq0/odim_archive/hsda-testing/snding_data/YBBN_20141127_00.nc'
radar_name       = 'CP2'

#field names (used to map to radar object fields)
fieldnames  = {'dbzh':'DBZ_CORR', 'zdr':'ZDR_CORR', 'rhv':'RHOHV', 'hca':'CLASS','hsda': 'CLASS_HSDA'}

#hsda vars
hca_hail_idx = [9] #index of hail classes in HCA
hca_hsda_idx = [11,12,13] #index to use for HSDA small, large and giant classes on top of HCA data
dzdr         = 0

#TODO
#Implement dzdr
#Implement beam blockage mapping

In [ ]:
#index vol files
vol_filelist = sorted(glob.glob(vol_path + '/*.mdv'))
for radar_file_name in vol_filelist:
    
    ###########################################################
    # Load file
    ###########################################################
    #load radar object
    if ".h5" in radar_file_name:
        radar = pyart.aux_io.read_odim_h5(radar_file_name)
    else:
        radar = pyart.io.read(radar_file_name)
    #extract date    
    date_str = radar.time['units'][-20:]
    dt       = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')    
    
    #fix field names
    radar.add_field('RHOHV', radar.fields.pop('cross_correlation_ratio'))
    radar.add_field('DBZH', radar.fields.pop('reflectivity'))
    radar.add_field('KDP', radar.fields.pop('specific_differential_phase'))
    radar.add_field('ZDR', radar.fields.pop('differential_reflectivity'))
    radar.add_field('PHIDP', radar.fields.pop('differential_phase'))
    radar.add_field('NCP', radar.fields.pop('normalized_coherent_power')) 
    
    ###########################################################
    # Filtering
    ###########################################################
    #rhohv gatefilter
    gatefilter = pyart.filters.GateFilter(radar)
    gatefilter.exclude_below('RHOHV', 0.7)
    
    #rhohv texture filtering
    #gatefilter = pyart.filters.moment_and_texture_based_gate_filter(
    
    ###########################################################
    # Correction
    ###########################################################
    
    #build temp information
    height, temperature, snr = dp_libs.snr_and_sounding(radar, sonde_ffn, refl_field_name='DBZH', temp_field_name="temp")
    radar.add_field('TEMPERATURE', temperature, replace_existing=True)
    radar.add_field('HEIGHT', height, replace_existing=True)
    radar.add_field('SNR', snr, replace_existing=True)
    
    #RHOHV Noise correct
    rho_corr = dp_libs.correct_rhohv(radar)
    radar.add_field_like('RHOHV', 'RHOHV_CORR', rho_corr, replace_existing=True)
    
    #ZDR Noise Correct
    corr_zdr = dp_libs.correct_zdr(radar)
    radar.add_field_like('ZDR', 'ZDR_CORR', corr_zdr, replace_existing=True)
    
    #unfold phidp
    phi_unfold = dp_libs.unfold_raw_phidp(radar, phi_name="PHIDP")
    radar.add_field("PHI_UNF", phi_unfold, replace_existing=True)

    #recalculate phidp
    phimeta, kdpmeta = dp_libs.phidp_bringi(radar, gatefilter, refl_field='DBZH', unfold_phidp_name="PHI_UNF")
    radar.add_field('PHIDP_BRINGI', phimeta, replace_existing=True)
    radar.add_field('KDP_BRINGI', kdpmeta, replace_existing=True)
    radar.fields['PHIDP_BRINGI']['long_name'] = "corrected_differential_phase"
    radar.fields['KDP_BRINGI']['long_name'] = "corrected_specific_differential_phase"

    ###########################################################
    # Attenuation
    ###########################################################
    
    #ZH attenuation correction
    atten_spec, zh_corr = dp_libs.correct_attenuation_zh_pyart(radar, refl_field='DBZH', rhv_field='RHOHV', phidp_field='KDP_BRINGI')
    radar.add_field('DBZ_CORR', zh_corr, replace_existing=True)
    radar.add_field('specific_attenuation_reflectivity', atten_spec, replace_existing=True)    
    
    #ZDR attenuation correction
    atten_spec_zdr, zdr_corr = dp_libs.correct_attenuation_zdr(radar, zdr_name='ZDR', kdp_name='KDP_BRINGI')
    radar.add_field_like('ZDR', 'ZDR_CORR', zdr_corr, replace_existing=True)
    radar.add_field('specific_attenuation_differential_reflectivity', atten_spec_zdr,
                    replace_existing=True)
    
    ###########################################################
    # Apply filter
    ###########################################################
    
    #apply rhohv filter
    radar.fields['DBZ_CORR']['data']   = dp_libs.filter_hardcoding(radar.fields['DBZ_CORR']['data'], gatefilter)
    radar.fields['ZDR_CORR']['data']   = dp_libs.filter_hardcoding(radar.fields['ZDR_CORR']['data'], gatefilter)
    radar.fields['KDP_BRINGI']['data'] = dp_libs.filter_hardcoding(radar.fields['KDP_BRINGI']['data'], gatefilter)
    radar.fields['RHOHV']['data']      = dp_libs.filter_hardcoding(radar.fields['RHOHV']['data'], gatefilter)
    
    ###########################################################
    # Classifications
    ###########################################################
    
    #CSU HCA
    hydro_class = dp_libs.hydrometeor_classification(radar, refl_name='DBZ_CORR', zdr_name='ZDR_CORR', kdp_name='KDP_BRINGI', rhohv_name='RHOHV', temperature_name='TEMPERATURE')
    radar.add_field('CLASS', hydro_class, replace_existing=True)    
    
    #HSDA
    hsda_data = hsda.main(radar,sonde_ffn,fieldnames,hca_hail_idx,hca_hsda_idx,dzdr)
    the_comments = "1: Drizzle; 2: Rain; 3: Ice Crystals; 4: Aggregates; " +\
                   "5: Wet Snow; 6: Vertical Ice; 7: LD Graupel; 8: HD Graupel; 9: NOT USED; 10: Big Drops" +\
                   "11: Small Hail (< 25 mm); 12: Large Hail (25 - 50 mm); 13: Giant Hail (> 50 mm)"
    hsda_meta   = {'data': hsda_data, 'units': ' ', 'long_name': 'Hydrometeor classification + HSDA',
                  'standard_name': 'Hydrometeor_ID_HSDA', 'comments': the_comments}
    radar.add_field('HSDA', hsda_meta, replace_existing=True) 
    
    
    #HDR
    hdr_data = hdr.main(radar,fieldnames)
    the_comments = "Applies the transform from hdr to mm used by Depue et al. 2009"
    hdr_meta     = {'data': hdr_data, 'units': 'mm', 'long_name': 'Hail Differential Reflectivity',
                  'standard_name': 'HDR', 'comments': the_comments}
    radar.add_field('HDR', hdr_meta, replace_existing=True)
    
    ###########################################################
    # CFradial output
    ###########################################################
    
    #write radar object to file
    out_fn  = '_'.join([radar_name, dt.strftime('%Y%m%d_%H%M%S'), 'processed']) + '.nc'
    out_ffn = '/'.join([out_path, out_fn])
    try:
        os.remove(out_ffn)
    except OSError:
        pass  
    pyart.io.write_cfradial(out_ffn, radar)
    
    print('completed volume ' + out_ffn)
    
    
    ###########################################################
    # Gridded Output
    ###########################################################
    
    out_fn  = '_'.join([radar_name, dt.strftime('%Y%m%d_%H%M%S'), 'meshgrids']) + '.nc'
    out_ffn = '/'.join([out_path,out_fn])
    try:
        os.remove(out_ffn)
    except OSError:
        pass
    #MESH
    mesh.main(radar, fieldnames, out_ffn, sonde_ffn)
    
    print('completed grid ' + out_ffn)
    
print('finished')

/home/548/jss548/dev/pyHail/dp_libs.py:237: RuntimeWarning: invalid value encountered in log10
  zdr = radar.fields[zdr_name]['data']
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyart/correct/phase_proc.py:188: RuntimeWarning: invalid value encountered in sqrt
  noise = smooth_and_trim(np.sqrt((line - signal) ** 2), window_len=wl)
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyart/correct/phase_proc.py:189: RuntimeWarning: divide by zero encountered in true_divide
  return abs(signal) / noise


completed volume /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_043602_processed.nc


/home/548/jss548/dev/pyHail/mesh.py:103: RuntimeWarning: divide by zero encountered in log
  POSH           = 29 * np.log(SHI/WT) + 50


completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_043602_meshgrids.nc
